In [17]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [18]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [14]:
sesh = tf.InteractiveSession()

In [2]:
x = tf.placeholder(tf.float32, [None, 784])
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [6]:
y = tf.matmul(x, W) + b

In [7]:
y_ = tf.placeholder(tf.float32, [None, 10])

In [8]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))

In [57]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(
    cross_entropy)

In [58]:
batch_size = 100
dataset = MNISTDataset('mnist', 100)

In [59]:
sesh.run(tf.global_variables_initializer())

In [70]:
for i in range(1000):
    batch = mnist.train.next_batch(batch_size)
    train_step.run(feed_dict={x: batch[0],
                              y_: batch[1]})

In [74]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

0.9115


## multi-layer

In [144]:
def weight_var(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_var(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

In [181]:
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

n_first_layer = 10
W_fc1 = tf.Variable(tf.zeros([784, n_first_layer]))
b_fc1 = bias_var([n_first_layer])
h_fc1 = tf.matmul(x, W_fc1) + b_fc1
# h_fc1 = tf.nn.relu(tf.matmul(x, W_fc1) + b_fc1)

W_fc2 = tf.Variable(tf.zeros([n_first_layer , 10]))
b_fc2 = bias_var([10])
y = tf.matmul(h_fc1, W_fc2) + b_fc2

In [182]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
        labels=y_, logits=y))
train_step = tf.train.GradientDescentOptimizer(0.1).minimize(
    cross_entropy)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [183]:
with tf.Session() as sesh:
    sesh.run(tf.global_variables_initializer())
    batch_size = 1000
    for i in range(1000):
        batch = mnist.train.next_batch(batch_size)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(
                feed_dict={x: batch[0], y_: batch[1]})
            print('step {}, training accuracy {}'.format(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0],
                                  y_: batch[1]})

step 0, training accuracy 0.0820000022649765
step 100, training accuracy 0.20600000023841858
step 200, training accuracy 0.26100000739097595
step 300, training accuracy 0.3089999854564667
step 400, training accuracy 0.34599998593330383
step 500, training accuracy 0.46299999952316284
step 600, training accuracy 0.5720000267028809
step 700, training accuracy 0.5989999771118164
step 800, training accuracy 0.6349999904632568
step 900, training accuracy 0.6240000128746033


## conv

In [19]:
def weight_var(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_var(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

def conv2d(x, W, strides=[1, 1, 1, 1]):
    return tf.nn.conv2d(x, W, strides=strides, padding='SAME')

def max_pool_2x2(x):
    return tf.nn.max_pool(x, ksize=[1, 2, 2, 1],
                        strides=[1, 2, 2, 1], padding='SAME')

In [20]:
# build computation graph
x = tf.placeholder(tf.float32, [None, 784])
y_ = tf.placeholder(tf.float32, [None, 10])

x_img = tf.reshape(x, [-1, 28, 28, 1])

strides = [1, 1, 1, 1]

W_conv1 = weight_var([5, 5, 1, 32])
b_conv1 = bias_var([32])
h_conv1 = tf.nn.relu(conv2d(x_img, W_conv1) + b_conv1)
h_pool1 = max_pool_2x2(h_conv1)

W_conv2 = weight_var([5, 5, 32, 64])
b_conv2 = bias_var([64])
h_conv2 = tf.nn.relu(conv2d(h_pool1, W_conv2) + b_conv2)
h_pool2 = max_pool_2x2(h_conv2)
h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])

W_fc1 = weight_var([7*7*64, 1024])
b_fc1 = bias_var([1024])
h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

keep_prob = tf.placeholder(tf.float32)
h_fc1_drop = tf.nn.dropout(h_fc1, keep_prob)

W_fc2 = weight_var([1024, 10])
b_fc2 = bias_var([10])

y = tf.matmul(h_fc1_drop, W_fc2) + b_fc2

In [21]:
cross_entropy = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=y_, logits=y))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [23]:
with tf.Session() as sesh:
    sesh.run(tf.global_variables_initializer())
    batch_size = 50
    for i in range(10000):
        batch = mnist.train.next_batch(batch_size)
        if i % 100 == 0:
            train_accuracy = accuracy.eval(
                feed_dict={x: batch[0], y_: batch[1], keep_prob: 1.})
            print('step {}, training accuracy {}'.format(i, train_accuracy))
        train_step.run(feed_dict={x: batch[0],
                                  y_: batch[1],
                                  keep_prob: 0.5})

step 0, training accuracy 0.07999999821186066
step 100, training accuracy 0.8399999737739563
step 200, training accuracy 0.8999999761581421
step 300, training accuracy 0.8999999761581421
step 400, training accuracy 0.8600000143051147
step 500, training accuracy 0.8999999761581421
step 600, training accuracy 1.0
step 700, training accuracy 0.9399999976158142
step 800, training accuracy 0.9399999976158142
step 900, training accuracy 0.9599999785423279
step 1000, training accuracy 0.9599999785423279
step 1100, training accuracy 0.9200000166893005
step 1200, training accuracy 0.9399999976158142
step 1300, training accuracy 0.9800000190734863
step 1400, training accuracy 0.9800000190734863
step 1500, training accuracy 1.0
step 1600, training accuracy 0.9800000190734863
step 1700, training accuracy 0.9599999785423279
step 1800, training accuracy 0.9800000190734863
step 1900, training accuracy 0.9800000190734863
step 2000, training accuracy 0.9800000190734863
step 2100, training accuracy 0.98

KeyboardInterrupt: 